## CNN model for MNIST dataset

### Prepare Models

In [1]:
import numpy as np
import torch
import tensorflow as tf

device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-02-06 01:35:47.705367: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 01:35:47.737674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 01:35:47.737699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 01:35:47.738354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 01:35:47.743043: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import numpy as np
import torch

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images_tf = train_images / 255.0
test_images_tf = test_images / 255.0
train_images_tf = train_images_tf.reshape(train_images.shape[0], 28, 28, 1)
test_images_tf = test_images_tf.reshape(test_images.shape[0], 28, 28, 1)

train_images_tf_14 = tf.image.resize(train_images_tf, [14, 14]).numpy()
test_images_tf_14 = tf.image.resize(test_images_tf, [14, 14]).numpy()


2024-02-06 01:35:50.214591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-06 01:35:50.215485: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-06 01:35:50.215575: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Convert to PyTorch format [batch_size, channels, height, width]
train_images_pt = torch.tensor(train_images_tf).permute(0, 3, 1, 2).float()
test_images_pt = torch.tensor(test_images_tf).permute(0, 3, 1, 2).float()


train_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()
test_images_pt_14 =  torch.tensor(test_images_tf_14).permute(0, 3, 1, 2).float()

## 5_11_80_10 CNN Model for 14x14 input and 3x3 kernel

In [41]:
layers = [14, 5, 11, 80, 10, 3]

# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(5, 3, activation='relu', input_shape=(layers[0], layers[0], 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(11, 3, activation='relu'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(80, activation = 'relu'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

model_tf.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 12, 12, 5)         50        
                                                                 
 average_pooling2d_8 (Avera  (None, 6, 6, 5)           0         
 gePooling2D)                                                    
                                                                 
 conv2d_9 (Conv2D)           (None, 4, 4, 11)          506       
                                                                 
 average_pooling2d_9 (Avera  (None, 2, 2, 11)          0         
 gePooling2D)                                                    
                                                                 
 flatten_4 (Flatten)         (None, 44)                0         
                                                                 
 dense_8 (Dense)             (None, 80)               

In [42]:
# Train the model
history = model_tf.fit(train_images_tf_14, train_labels, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.6917 - accuracy: 0.7884 - val_loss: 0.3224 - val_accuracy: 0.9048
Epoch 2/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.3197 - accuracy: 0.9026 - val_loss: 0.2002 - val_accuracy: 0.9420
Epoch 3/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2235 - accuracy: 0.9329 - val_loss: 0.1552 - val_accuracy: 0.9603
Epoch 4/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1760 - accuracy: 0.9468 - val_loss: 0.1343 - val_accuracy: 0.9618
Epoch 5/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1513 - accuracy: 0.9539 - val_loss: 0.1148 - val_accuracy: 0.9658
Epoch 6/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1346 - accuracy: 0.9600 - val_loss: 0.1125 - val_accuracy: 0.9660
Epoch 7/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1222 - accuracy: 0.9626 - val_loss: 0.1009 - val_accuracy:

In [43]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf_14, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.0971 - accuracy: 0.9707 - 218ms/epoch - 697us/step

Test accuracy: 0.9707000255584717


### Convert it to Pytorch

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

layers = [14, 5, 11, 80, 10, 3]
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, layers[1], layers[-1]) 
        self.conv2 = nn.Conv2d(layers[1], layers[2], layers[-1]) 

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(11 * 2 * 2, layers[3]) # 256 * 120
        self.fc2 = nn.Linear(layers[3], layers[4])

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.relu(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.relu(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),layers[2],-1)  # 16 output channels
        x = np.transpose(x, (0,2,1)).reshape(batch_size,-1)
        #x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation function here, will use CrossEntropyLoss later
        return x

model_pt = Net()

In [45]:
# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [46]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images_tf_14[189:190]  # Reshape to (1, 784) for DNN
controlled_input_pt = test_images_pt_14[189:190]

# Test TensorFlow Model
output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)

# Test PyTorch Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)
print("PyTorch Basic Model Output:", output_pt)

1/1 [==============================] - 0s 47ms/step
TensorFlow Basic Model Output: [[-4.868403   4.1331396 -4.9311266 -1.1776102 -0.9888911 -2.0313888
  -4.0539174 -3.8575764  0.7993514 -0.0939117]]
PyTorch Basic Model Output: tensor([[-4.8684,  4.1331, -4.9311, -1.1776, -0.9889, -2.0314, -4.0539, -3.8576,
          0.7994, -0.0939]])


In [47]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming test_labels are already loaded
test_dataset = TensorDataset(test_images_pt_14, torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model_pt, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 97.07%


In [48]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=256):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [49]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images_tf_14)
predictions_pt = get_predictions_pt(model_pt, test_images_pt_14)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images)} samples")

8/8 [==============================] - 0s 618us/step


1/1 [==============================] - 0s 10ms/step
Number of mismatches: 0 out of 10000 samples


### Save Model

In [50]:
layers

[14, 5, 11, 80, 10, 3]

In [51]:
import os
# Tensorflow
arch_folder = "./input-conv2d-conv2d-dense-dense/"
os.makedirs(arch_folder, exist_ok=True)

model_name = "_".join([str(x) for x in layers])
model_tf.save(arch_folder + model_name + '.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open(arch_folder + model_name + '.tflite', 'wb') as f:
    f.write(tflite_model)


/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /tmp/tmp577cc43o/assets


INFO:tensorflow:Assets written to: /tmp/tmp577cc43o/assets
2024-02-06 02:54:22.909645: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-06 02:54:22.909665: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-06 02:54:22.909809: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp577cc43o
2024-02-06 02:54:22.910631: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-06 02:54:22.910641: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp577cc43o
2024-02-06 02:54:22.912794: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-06 02:54:22.951124: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp577cc43o
2024-02-06 02:54:22.960978: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [52]:
# Save entire model
torch.save(model_pt, arch_folder + model_name + ".pt")
# Save only the state_dict
torch.save(model_pt.state_dict(), arch_folder + model_name + ".pth")
with torch.no_grad():
    torch.onnx.export(model_pt, controlled_input_pt, arch_folder + model_name + ".onnx")

## 784_6_16_10 Conv2d-Conv2D-Dense

### Tensorflow Model

In [3]:
# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Train the model
history = model_tf.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.1)


2024-02-05 16:20:13.152800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 16:20:13.153747: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 16:20:13.153840: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/10


2024-02-05 16:20:14.150712: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-05 16:20:14.190407: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-05 16:20:14.286295: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1af0a13d90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-05 16:20:14.286314: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-02-05 16:20:14.291426: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-05 16:20:14.324180: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
I0000 00:00:1707121214.359937 2876481 device_compiler.

1688/1688 [==============================] - 3s 1ms/step - loss: 0.3205 - accuracy: 0.9070 - val_loss: 0.1176 - val_accuracy: 0.9668
Epoch 2/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1153 - accuracy: 0.9645 - val_loss: 0.0852 - val_accuracy: 0.9767
Epoch 3/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0844 - accuracy: 0.9742 - val_loss: 0.0595 - val_accuracy: 0.9833
Epoch 4/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0684 - accuracy: 0.9791 - val_loss: 0.0558 - val_accuracy: 0.9840
Epoch 5/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0584 - accuracy: 0.9821 - val_loss: 0.0479 - val_accuracy: 0.9868
Epoch 6/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0518 - accuracy: 0.9842 - val_loss: 0.0549 - val_accuracy: 0.9848
Epoch 7/10
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0463 - accuracy: 0.9856 - val_loss: 0.0473 - val_accuracy: 0.9863
Epo

In [4]:
model_tf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 12, 12, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 4, 4, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 10)                2

In [5]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.0387 - accuracy: 0.9877 - 194ms/epoch - 621us/step

Test accuracy: 0.9876999855041504


### Convert to Pytorch

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(16 * 4 * 4, 10) # 256 * 120

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.relu(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.relu(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),16,-1)  # 16 output channels
        x = np.transpose(x, (0,2,1)).reshape(batch_size,-1)
        #x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = self.fc1(x)  # No activation function here, will use CrossEntropyLoss later
        return x
    

model_pt = Net()

In [7]:
# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))


In [8]:

# Select the image for TensorFlow
controlled_input_tf = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

In [9]:
# Test PyTorch Basic Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)

output_tf = model_tf.predict(controlled_input_tf) 
print("TF Basic Model Output:", output_tf)
print("PT Basic Model Output:", output_pt.cpu().numpy())

1/1 [==============================] - 0s 78ms/step
TF Basic Model Output: [[ -7.848706   -3.3647184   3.2896788  -2.3894203 -20.845005   -9.981148
  -20.85786    11.181317  -11.737088  -10.418606 ]]
PT Basic Model Output: [[ -7.8487053  -3.3647187   3.2896793  -2.3894205 -20.845015   -9.981146
  -20.857857   11.18132   -11.737091  -10.418609 ]]


In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming the TensorFlow MNIST data has already been loaded
# Convert test_images to PyTorch tensor and permute
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# Assuming test_labels are already loaded
test_dataset = TensorDataset(test_images_pt, torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model_pt, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 98.77%


In [11]:
def get_predictions_tf(model, test_images, batch_size=256):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=256):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [13]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images)} samples")


1/1 [==============================] - 0s 13ms/step
Number of mismatches: 0 out of 10000 samples


#### Save Models for 784_6_16_10

In [14]:
import os
# Tensorflow
arch_folder = "./input-conv2d-conv2d-dense/"
os.makedirs(arch_folder, exist_ok=True)

model_name = "784_6_16_10"
model_tf.save(arch_folder + model_name + '.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open(arch_folder + model_name + '.tflite', 'wb') as f:
    f.write(tflite_model)


/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /tmp/tmpdq8sxxdi/assets


INFO:tensorflow:Assets written to: /tmp/tmpdq8sxxdi/assets
2024-02-05 16:22:44.382112: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-05 16:22:44.382125: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-05 16:22:44.382279: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpdq8sxxdi
2024-02-05 16:22:44.382902: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-05 16:22:44.382907: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpdq8sxxdi
2024-02-05 16:22:44.384357: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-02-05 16:22:44.384799: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-05 16:22:44.406560: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [15]:
# pytorch

# Save entire model
torch.save(model_pt, arch_folder + model_name + ".pt")
# Save only the state_dict
torch.save(model_pt.state_dict(), arch_folder + model_name + ".pth")
with torch.no_grad():
    torch.onnx.export(model_pt, controlled_input_pt, arch_folder + model_name + ".onnx")

## 784_6_16_120_84_10 Conv2d-Conv2D-Dense-Dense-Dense

### Prepare Model

In [16]:
# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Train the model
history = model_tf.fit(train_images, train_labels, epochs=10, batch_size=256, validation_split=0.1)


Epoch 1/10
211/211 [==============================] - 2s 3ms/step - loss: 0.5725 - accuracy: 0.8337 - val_loss: 0.1971 - val_accuracy: 0.9423
Epoch 2/10
211/211 [==============================] - 0s 2ms/step - loss: 0.1843 - accuracy: 0.9451 - val_loss: 0.1256 - val_accuracy: 0.9608
Epoch 3/10
211/211 [==============================] - 0s 1ms/step - loss: 0.1163 - accuracy: 0.9645 - val_loss: 0.0845 - val_accuracy: 0.9753
Epoch 4/10
211/211 [==============================] - 0s 2ms/step - loss: 0.0914 - accuracy: 0.9727 - val_loss: 0.0739 - val_accuracy: 0.9783
Epoch 5/10
211/211 [==============================] - 0s 2ms/step - loss: 0.0750 - accuracy: 0.9767 - val_loss: 0.0683 - val_accuracy: 0.9818
Epoch 6/10
211/211 [==============================] - 0s 2ms/step - loss: 0.0659 - accuracy: 0.9797 - val_loss: 0.0577 - val_accuracy: 0.9842
Epoch 7/10
211/211 [==============================] - 0s 2ms/step - loss: 0.0597 - accuracy: 0.9821 - val_loss: 0.0596 - val_accuracy: 0.9842
Epoch 

In [17]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.0377 - accuracy: 0.9872 - 247ms/epoch - 790us/step

Test accuracy: 0.9872000217437744


In [18]:
model_tf.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d_2 (Avera  (None, 12, 12, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_3 (Avera  (None, 4, 4, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 120)              

### Convert to Pytorch Model

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(16 *4 * 4,120) # 256 * 120
        self.fc2 = nn.Linear(120, 84)         # 120 inputs, 84 outputs
        self.fc3 = nn.Linear(84, 10)          # 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.relu(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.relu(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),16,-1)  # 16 output channels
        x = np.transpose(x, (0,2,1)).reshape(batch_size,-1)
        #x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # No activation function here, will use CrossEntropyLoss later
        return x
    

model_pt = Net()

In [20]:
# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the third dense layer (fc3) from model_tf to model_pt
weights, biases = model_tf.layers[7].get_weights()
model_pt.fc3.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc3.bias = nn.Parameter(torch.from_numpy(biases))

In [21]:
# Select the image for TensorFlow
controlled_input_tf = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

In [22]:
# Test PyTorch Basic Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)

output_tf = model_tf.predict(controlled_input_tf) 
print("TF Basic Model Output:", output_tf)
print("PT Basic Model Output:", output_pt.cpu().numpy())

1/1 [==============================] - 0s 47ms/step


TF Basic Model Output: [[ -5.0148506   -1.157651     3.4304547   -0.13103871 -10.998174
   -7.736541   -10.994179    10.4635515   -2.6589053   -6.6241155 ]]
PT Basic Model Output: [[ -5.0148487   -1.1576512    3.430455    -0.13104191 -10.998172
   -7.7365384  -10.994177    10.463552    -2.6589077   -6.624114  ]]


In [23]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming the TensorFlow MNIST data has already been loaded
# Convert test_images to PyTorch tensor and permute
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# Assuming test_labels are already loaded
test_dataset = TensorDataset(test_images_pt, torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model_pt, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 98.72%


In [24]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images)} samples")


1/1 [==============================] - 0s 9ms/step
Number of mismatches: 0 out of 10000 samples


#### Save Model for 784_6_16_120_84_10

In [25]:
import os
# Tensorflow
arch_folder = "./input-conv2d-conv2d-dense-dense-dense/"
os.makedirs(arch_folder, exist_ok=True)

model_name = "784_6_16_120_84_10"
model_tf.save(arch_folder + model_name + '.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open(arch_folder + model_name + '.tflite', 'wb') as f:
    f.write(tflite_model)


/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /tmp/tmptka8q2hl/assets


INFO:tensorflow:Assets written to: /tmp/tmptka8q2hl/assets
2024-02-05 16:24:02.657200: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-05 16:24:02.657221: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-05 16:24:02.657370: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmptka8q2hl
2024-02-05 16:24:02.658461: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-05 16:24:02.658471: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmptka8q2hl
2024-02-05 16:24:02.661524: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-05 16:24:02.707050: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmptka8q2hl
2024-02-05 16:24:02.718252: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [26]:
# pytorch

# Save entire model
torch.save(model_pt, arch_folder + model_name + ".pt")
# Save only the state_dict
torch.save(model_pt.state_dict(), arch_folder + model_name + ".pth")
with torch.no_grad():
    torch.onnx.export(model_pt, controlled_input_pt, arch_folder + model_name + ".onnx")